In [1]:
from hashlib import sha256
import numpy as np
import struct
from datetime import datetime

In [2]:
class block_header():
    def __init__(self, contents_length, contents_hash, previous_hash, nonce):
        self.contents_length = contents_length  
        self.contents_hash = contents_hash
        self.previous_hash = previous_hash
        self.nonce = nonce
        
    def pack(self):
        format_str = '<I32s32sI'
        packed_data = struct.pack(format_str,
                                  self.contents_length,
                                  self.contents_hash,
                                  self.previous_hash,
                                  self.nonce)
        return packed_data


In [3]:
def str_to_byte(s):
    s = [ord(i) for i in s] + [0]
    s = ''.join(['{0:08b}'.format(i) for i in s])
    return int(s, 2).to_bytes((len(s) + 7) // 8, byteorder='big')


def build_block(previous, contents, length):
    if previous is None:
        previous_hash = bytearray(np.zeros(32, dtype=np.uint8))
    else:
        buffer = bytearray(256)
        packed_data = previous.pack()
        buffer[:len(packed_data)] = packed_data
        previous_hash = sha256( buffer[:len(packed_data)] ).digest()
        
    content_hash = sha256(str_to_byte(line_buffer)).digest()    
    header = block_header(length, content_hash, previous_hash, 0 )
    return header

def calc_sha_256(header, buffer=None):
    if buffer is None:
        buffer = bytearray(256)
    packed_data = header.pack()
    buffer[:len(packed_data)] = packed_data
    block_hash = sha256( buffer[:len(packed_data)] ).digest()
    return block_hash

def pow(header, target):
    for i in range( 2**32 -1):
        header.nonce = i
        packed_data = header.pack()
        buffer[:len(packed_data)] = packed_data
        block_hash = sha256( buffer[:len(packed_data)] ).digest()
        if block_hash < target:
            return

In [4]:
target = bytearray(np.zeros(32, dtype=np.uint8))
target[2] = 0xff
target

bytearray(b'\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [5]:
buffer = bytearray(np.zeros(32, dtype=np.uint8))

# Hello world blockchain

In [6]:
LINE_MAX = 4096

block_no = 0
previous = None

line_buffer = "Hello world"

size = len(line_buffer) + 1

print("creating block %i: " % block_no, end=' ')
print(line_buffer)

header = build_block(previous, line_buffer, size)

start = datetime.now()
pow(header,target)
print( 'Time spent(ms):',(datetime.now()-start).total_seconds()*1000 )

previous = header

block_no += 1

print("previous: ", previous.previous_hash.hex())
print("contents: ", previous.contents_hash.hex())
print("nonce: ", previous.nonce)

test_hash = calc_sha_256(header)
print("hash: ", test_hash.hex())

creating block 0:  Hello world
Time spent(ms): 1163.299
previous:  0000000000000000000000000000000000000000000000000000000000000000
contents:  27518ba9683011f6b396072c05f6656d04f5fbc3787cf92490ec606e5092e326
nonce:  58450
hash:  00006cccf4ec1222f42a38b284ff729d20201dfea8ff7d7f677dc0898380dbfc


# Blockchain of a file

In [7]:
LINE_MAX = 4096

block_no = 0
previous = None

with open("gorgias_trunc.txt", "r", encoding="ascii") as file:
    for line in file:
        line_buffer = line.strip()#[:-1]
        size = len(line_buffer) + 1

        print("creating block %i: " % block_no, end=' ')
        print(line_buffer)

        header = build_block(previous, line_buffer, size)

        start = datetime.now()
        pow(header,target)
        print( 'Time spent(ms):',(datetime.now()-start).total_seconds()*1000 )

        previous = header

        block_no += 1

        print("previous: ", previous.previous_hash.hex())
        print("contents: ", previous.contents_hash.hex())
        print("nonce: ", previous.nonce)

        test_hash = calc_sha_256(header)
        print("hash: ", test_hash.hex())
        print()

creating block 0:  The Project Gutenberg EBook of Gorgias, by Plato
Time spent(ms): 2815.037
previous:  0000000000000000000000000000000000000000000000000000000000000000
contents:  f3a47d06620d74467cceb09de6ce3a5a293381d85aa8570dfda11796baacfda6
nonce:  142327
hash:  000068754d36ea13ffdbd5041d26b0a0a4ff0fbb918e47bee26d01b3afb7803e

creating block 1:  
Time spent(ms): 2368.1369999999997
previous:  000068754d36ea13ffdbd5041d26b0a0a4ff0fbb918e47bee26d01b3afb7803e
contents:  6e340b9cffb37a989ca544e6bb780a2c78901d3fb33738768511a30617afa01d
nonce:  120404
hash:  00004ca122d4e638adb6fdcbc58727a75e2a1327a459f6159c882a9423457022

creating block 2:  This eBook is for the use of anyone anywhere at no cost and with
Time spent(ms): 3589.567
previous:  00004ca122d4e638adb6fdcbc58727a75e2a1327a459f6159c882a9423457022
contents:  941b1a175dfd243e0caf5bf0115712ba95f583da49089a1e672899fc84b8de32
nonce:  183528
hash:  00001c7e33a8bdbd643a3e5e313b7278bf9f7c999e580ecfc61108e4839597af

creating block 3:  almo

Time spent(ms): 2199.783
previous:  00008def17c0ad49c88858ad377bafb53bda875ca51dd9e44dcf8fad616e7ba3
contents:  6e340b9cffb37a989ca544e6bb780a2c78901d3fb33738768511a30617afa01d
nonce:  112208
hash:  000072bfb378de99ab05bd51d1feffb1e98a0e748e21d103ebf28d1f320ae106

creating block 28:  
Time spent(ms): 1546.061
previous:  000072bfb378de99ab05bd51d1feffb1e98a0e748e21d103ebf28d1f320ae106
contents:  6e340b9cffb37a989ca544e6bb780a2c78901d3fb33738768511a30617afa01d
nonce:  78796
hash:  000036a07960b58ec67c578c4681d3bed74414b68a914240888210688452c543

creating block 29:  
Time spent(ms): 1146.818
previous:  000036a07960b58ec67c578c4681d3bed74414b68a914240888210688452c543
contents:  6e340b9cffb37a989ca544e6bb780a2c78901d3fb33738768511a30617afa01d
nonce:  58329
hash:  00009bf008b7fa258ce9832453637b6046955717964cbe976a95cbb88e866412

creating block 30:  
Time spent(ms): 1957.539
previous:  00009bf008b7fa258ce9832453637b6046955717964cbe976a95cbb88e866412
contents:  6e340b9cffb37a989ca544e6bb780a2c

Time spent(ms): 2224.831
previous:  000042607a2814180875103aa571dce32325ddf47a3d0ecf8c5ea8d62235a66e
contents:  88a90dcfb80dd925550cc4a064680a8968f4258c518a45f66bc604f7ff5f4116
nonce:  114127
hash:  00009e3422d150ae82e24e75eb8ce11fc62bab9008be271d873bd5f1c51f0fed

